In [43]:
import pandas as pd
import numpy as np
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# Set random seed for reproducibility
np.random.seed(42)

# ============================================================================
# FILE PATHS - EASILY CONFIGURABLE
# ============================================================================

INPUT_FILE = "/Users/matt.fritz/Desktop/SFDC Won Data FY16 pivot.csv"
OUTPUT_FILE = "/Users/matt.fritz/Desktop/quarterly_forecasts_fy26_fy29.csv"

# ============================================================================
# CONFIGURATION - ALL TEAM PARAMETERS IN ONE PLACE
# ============================================================================

team_config = {
    'CorpFoundation': {
        # FY26 Q1 actual (already in historical data)
        'fy26_q1_actual': 23_499_210,  # From your data
        
        # FY26 Q2-Q4 assumptions (you input these based on your info)
        'fy26_q2_forecast': 21_500_000,  # Oct-Dec 2025 - YOUR INPUT
        'fy26_q3_forecast': 14_000_000,  # Jan-Mar 2026 - YOUR INPUT
        'fy26_q4_forecast':  8_200_000,  # Apr-Jun 2026 - YOUR INPUT
        
        # Growth rate parameters
        'decay_factor': 0.925,         # Weight recent history
        'volatility_dampen': 0.2,      # Reduce historical volatility
        'growth_cap': None,            # No cap on growth
        'use_last_n_yoy': 24,          # Number of quarterly yoys
        
        # Std dev parameters
        'std_dev_multiplier': 0.8,    # Standard volatility
        
        # Seasonality (Q1-Q4 distribution)
        'seasonality': [0.23, 0.25, 0.26, 0.26]
    },
    
    'Marketplace': {
        'fy26_q1_actual': 25_121_098,
        'fy26_q2_forecast': 22_000_000,  
        'fy26_q3_forecast': 13_500_000,  
        'fy26_q4_forecast': 13_800_000,  
        'decay_factor': 0.25,
        'volatility_dampen': 0.1,
        'growth_cap': None,
        'use_last_n_yoy': 24,
        'std_dev_multiplier': 0.7,
        'seasonality': [0.24, 0.25, 0.25, 0.26]
    },
    
    'MajorGifts': {
        'fy26_q1_actual': 810_000,
        'fy26_q2_forecast': 1_600_000, 
        'fy26_q3_forecast': 1_400_000,  
        'fy26_q4_forecast': 1_700_000,  
        'growth_cap': 80,
        'use_special_model': True,
        'std_dev_multiplier': 1.5,
        'seasonality': [0.20, 0.25, 0.25, 0.30]
    },
    
    'Government': {
        'fy26_q1_actual': 3_000_000,
        'fy26_q2_forecast': 6_900_000,  
        'fy26_q3_forecast': 1_800_000,  
        'fy26_q4_forecast': 2_000_000,  
        'growth_cap': 80,
        'use_special_model': True,
        'std_dev_multiplier': 2.0,
        'seasonality': [0.15, 0.20, 0.35, 0.30]
    }
}

# ============================================================================
# DATA LOADING AND HISTORICAL ANALYSIS
# ============================================================================

def load_historical_data(filepath):
    """
    Load historical data from CSV file
    Expected columns: Year, Quarter, CorpFoundation, Government, MajorGifts, Marketplace,
                      CorpFoundation_YOY%, Government_YOY%, MajorGifts_YOY%, Marketplace_YOY%
    """
    try:
        df = pd.read_csv(filepath)
        print(f"Successfully loaded data from {filepath}")
        print(f"Shape: {df.shape}")
        return df
    except Exception as e:
        print(f"Error loading file: {e}")
        raise

def calculate_rolling_ttm_yoy(quarterly_data, team_column, fy26_assumptions=None):
    """
    Calculate rolling TTM YoY rates including FY26 assumptions
    
    TTM = Trailing Twelve Months (sum of 4 quarters)
    YoY = Current TTM / Year-ago TTM - 1
    """
    # Create a working dataframe
    df = quarterly_data[['Year', 'Quarter', team_column]].copy()
    df = df.rename(columns={team_column: 'Amount'})
    
    # Add FY26 assumptions if provided
    if fy26_assumptions:
        new_rows = []
        for q, amount in fy26_assumptions.items():
            if q in [2, 3, 4]:  # Q2-Q4
                new_rows.append({
                    'Year': 2026,
                    'Quarter': q,
                    'Amount': amount
                })
        
        if new_rows:
            df = pd.concat([df, pd.DataFrame(new_rows)], ignore_index=True)
    
    # Sort by year and quarter
    df = df.sort_values(['Year', 'Quarter'])
    
    # Calculate TTM for each quarter
    df['TTM'] = df['Amount'].rolling(window=4, min_periods=4).sum()
    
    # Calculate YoY growth
    df['TTM_YoY'] = (df['TTM'] / df['TTM'].shift(4) - 1) * 100
    
    # Extract YoY rates (excluding NaN)
    yoy_rates = df['TTM_YoY'].dropna().values
    
    return yoy_rates, df

def calculate_historical_metrics(df, team_column, config):
    """
    Calculate historical YoY rates and quarterly volatility for a team
    Including FY26 assumptions to get more recent YoY data points
    """
    # Extract FY26 assumptions from config
    fy26_assumptions = {}
    for q in [2, 3, 4]:
        key = f'fy26_q{q}_forecast'
        if key in config:
            fy26_assumptions[q] = config[key]
    
    # Calculate YoY with FY26 assumptions included
    yoy_rates, ttm_df = calculate_rolling_ttm_yoy(df, team_column, fy26_assumptions)
    
    # Calculate quarterly coefficient of variation (excluding FY26 projections for volatility)
    historical_amounts = df[team_column].dropna()
    
    if len(historical_amounts) >= 4:
        # Group by quarter to get seasonal volatility
        df_temp = df[df[team_column].notna()].copy()
        quarterly_stats = df_temp.groupby('Quarter')[team_column].agg(['mean', 'std'])
        quarterly_cv = (quarterly_stats['std'] / quarterly_stats['mean']).mean()
    else:
        quarterly_cv = 0.15  # Default 15% if insufficient data
    
    # Get the last few YoY rates (these will have highest weight)
    last_n = min(16, len(yoy_rates))
    
    print(f"\n  YoY rates calculated: {len(yoy_rates)} total")
    if len(yoy_rates) > 0:
        print(f"  Most recent YoY rates: {[f'{x:.0f}%' for x in yoy_rates[-last_n:]]}")
        print(f"  Note: Last 3 rates include FY26 assumptions" if fy26_assumptions else "")
    
    return {
        'yoy_rates': yoy_rates,
        'quarterly_cv': quarterly_cv,
        'ttm_dataframe': ttm_df
    }

# ============================================================================
# CORE FORECASTING FUNCTIONS
# ============================================================================

def generate_growth_rate(
    historical_yoy,
    decay_factor,
    volatility_dampen,
    growth_cap=None,
    use_last_n_yoy=16
):
    """
    Generate annual growth rate based on weighted historical YoY.

    - Uses up to the last N YoY values (default = 16)
    - Recent values receive exponentially higher weight per decay_factor
    - Applies volatility dampening and optional growth cap
    """
    if len(historical_yoy) == 0:
        return np.random.normal(5, 10)

    # Use only the most recent N YoY rates (if available)
    hist = historical_yoy[-use_last_n_yoy:] if len(historical_yoy) > use_last_n_yoy else historical_yoy

    # Calculate weighted average with exponential decay
    n = len(hist)
    weights = np.array([decay_factor ** (n - i - 1) for i in range(n)])
    weights /= weights.sum()

    weighted_mean = np.sum(hist * weights)
    historical_std = np.std(hist) if n > 1 else 15

    # Draw a growth rate around the weighted mean
    growth_rate = np.random.normal(weighted_mean, historical_std * volatility_dampen)

    # Enforce growth cap if defined
    if growth_cap is not None:
        growth_rate = min(growth_rate, growth_cap)

    return growth_rate


def generate_growth_rate_stochastic(historical_yoy, n_sim=10_000, summary='mean', cap=None):
    """
    Monte Carlo version returning a summary of simulated growth rates.
    summary ∈ {'mean', 'median', 'p50', 'p90', 'p10'}
    cap → optional numeric ceiling applied to the final result
    """
    sims = []

    # --- Simulation loop ----------------------------------------------------
    for _ in range(n_sim):
        if len(historical_yoy) < 3:
            val = np.random.choice(
                [-50, -20, 0, 30, 80, 150],
                p=[0.1, 0.2, 0.2, 0.2, 0.2, 0.1]
            )
        else:
            p20, p50, p80 = np.percentile(historical_yoy, [20, 50, 80])
            rand = np.random.random()

            if rand < 0.2:
                val = np.random.uniform(min(p20, -50), p50)
            elif rand < 0.8:
                val = np.random.uniform(0, p80)
            else:
                val = np.random.uniform(p80, min(200, p80 * 1.5))
        sims.append(val)

    # --- Summarize the simulated distribution -------------------------------
    sims = np.array(sims)
    if summary == 'mean':
        result = sims.mean()
    elif summary in ['median', 'p50']:
        result = np.percentile(sims, 50)
    elif summary == 'p90':
        result = np.percentile(sims, 90)
    elif summary == 'p10':
        result = np.percentile(sims, 10)
    else:
        raise ValueError(f"Unsupported summary type: {summary}")

    # --- Apply growth cap if provided ---------------------------------------
    if cap is not None:
        result = min(result, cap)

    return result

def calculate_quarterly_std(base_forecast, historical_cv, std_dev_multiplier, time_years):
    """
    Calculate quarterly standard deviation
    """
    base_uncertainty = historical_cv
    time_factor = 1 + (time_years * 0.2)
    std_dev = base_forecast * base_uncertainty * std_dev_multiplier * time_factor
    return std_dev

# ============================================================================
# MAIN FORECASTING FUNCTION
# ============================================================================

def generate_quarterly_forecasts(team_name, config, historical_metrics):
    """
    Generate quarterly forecasts with std dev through FY29 Q4
    """
    forecasts = []
    
    # Extract parameters
    seasonality = config['seasonality']
    historical_yoy = historical_metrics['yoy_rates']
    quarterly_cv = historical_metrics['quarterly_cv']
    
    # FY26 Q2-Q4 are user inputs (not generated)
    for quarter in [2, 3, 4]:
        forecast_amt = config[f'fy26_q{quarter}_forecast']
        std_dev = calculate_quarterly_std(
            forecast_amt,
            quarterly_cv,
            config['std_dev_multiplier'],
            time_years=0
        )
        
        forecasts.append({
            'Team': team_name,
            'FiscalYear': 2026,
            'Quarter': quarter,
            'Forecast': forecast_amt,
            'StdDev': std_dev
        })
    
    # Calculate FY26 total from actuals + forecasts
    fy26_total = (config['fy26_q1_actual'] + 
                  config['fy26_q2_forecast'] + 
                  config['fy26_q3_forecast'] + 
                  config['fy26_q4_forecast'])
    
    # Generate growth rates for FY27-FY29
    annual_growth_rates = []
    
    for year_idx in range(3):
        if config.get('use_special_model'):  
            growth_rate = generate_growth_rate_stochastic(
                historical_yoy,
                n_sim=10_000,
                summary='mean',
                cap=config.get('growth_cap')
            )        
        else:  # Standard model
            growth_rate = generate_growth_rate(
                historical_yoy,
                config['decay_factor'],
                config['volatility_dampen'],
                config.get('growth_cap'),
                config.get('use_last_n_yoy', 16)
            )
        annual_growth_rates.append(growth_rate)
    
    # Apply growth and generate quarterly forecasts
    annual_base = fy26_total
    
    for year_idx, fiscal_year in enumerate([2027, 2028, 2029]):
        # Apply annual growth
        annual_base = annual_base * (1 + annual_growth_rates[year_idx] / 100)
        
        # Distribute to quarters with seasonality
        for quarter in range(1, 5):
            forecast_amt = annual_base * seasonality[quarter - 1]
            
            # Add small random variation for realism
            forecast_amt *= np.random.normal(1, 0.02)
            
            std_dev = calculate_quarterly_std(
                forecast_amt,
                quarterly_cv,
                config['std_dev_multiplier'],
                time_years=fiscal_year - 2026
            )
            
            forecasts.append({
                'Team': team_name,
                'FiscalYear': fiscal_year,
                'Quarter': quarter,
                'Forecast': forecast_amt,
                'StdDev': std_dev,
                'AnnualGrowth': annual_growth_rates[year_idx]
            })
    
    return pd.DataFrame(forecasts), annual_growth_rates

# ============================================================================
# MAIN EXECUTION
# ============================================================================

def run_forecast_model():
    """
    Main function to run the complete forecasting model
    """
    print("="*80)
    print("YOY-BASED QUARTERLY FORECASTING MODEL WITH FY26 ASSUMPTIONS")
    print("="*80)
    print(f"\nInput file: {INPUT_FILE}")
    print(f"Output file: {OUTPUT_FILE}")
    
    # Load historical data
    print("\nLoading historical data...")
    df = load_historical_data(INPUT_FILE)
    
    all_forecasts = []
    
    for team_name, config in team_config.items():
        print(f"\n{team_name}")
        print("-"*60)
        
        # Map team names to column names
        column_mapping = {
            'CorpFoundation': 'CorpFoundation',
            'Marketplace': 'Marketplace',
            'MajorGifts': 'MajorGifts',
            'Government': 'Government'
        }
        
        team_column = column_mapping[team_name]
        
        # Calculate historical metrics including FY26 assumptions
        historical_metrics = calculate_historical_metrics(df, team_column, config)
        
        print(f"  Historical quarterly CV: {historical_metrics['quarterly_cv']:.1%}")
        
        # Show FY26 assumptions
        print(f"\n  FY26 Assumptions:")
        print(f"    Q1 (actual): ${config['fy26_q1_actual']/1e6:.1f}M")
        print(f"    Q2 (input):  ${config['fy26_q2_forecast']/1e6:.1f}M")
        print(f"    Q3 (input):  ${config['fy26_q3_forecast']/1e6:.1f}M")
        print(f"    Q4 (input):  ${config['fy26_q4_forecast']/1e6:.1f}M")
        
        # Generate forecasts
        team_forecasts, growth_rates = generate_quarterly_forecasts(
            team_name, config, historical_metrics
        )
        
        print(f"\n  Generated growth rates for FY27-29: {[f'{x:.0f}%' for x in growth_rates]}")
        
        # Calculate weighted mean for reference
        if len(historical_metrics['yoy_rates']) > 0:
            n = len(historical_metrics['yoy_rates'])
            weights = np.array([config['decay_factor'] ** (n - i - 1) for i in range(n)])
            weights = weights / weights.sum()
            weighted_mean = np.sum(historical_metrics['yoy_rates'] * weights)
            print(f"  Weighted historical YoY mean: {weighted_mean:.1f}%")
        
        all_forecasts.append(team_forecasts)
    
    # Combine all forecasts
    forecast_df = pd.concat(all_forecasts, ignore_index=True)
    
    # Add FY26 Q1 actuals
    actuals = []
    for team_name, config in team_config.items():
        actuals.append({
            'Team': team_name,
            'FiscalYear': 2026,
            'Quarter': 1,
            'Forecast': config['fy26_q1_actual'],
            'StdDev': 0,
            'AnnualGrowth': 0
        })
    
    actuals_df = pd.DataFrame(actuals)
    forecast_df = pd.concat([actuals_df, forecast_df], ignore_index=True)
    forecast_df = forecast_df.sort_values(['Team', 'FiscalYear', 'Quarter'])
    
    # Display summary
    print("\n" + "="*80)
    print("FORECAST SUMMARY")
    print("="*80)
    
    for team in team_config.keys():
        team_df = forecast_df[forecast_df['Team'] == team]
        
        print(f"\n{team}")
        print("-"*40)
        
        # Show all FY26 quarters
        print("  FY26 (Q1 actual, Q2-Q4 input):")
        for q in range(1, 5):
            row = team_df[(team_df['FiscalYear'] == 2026) & (team_df['Quarter'] == q)]
            if len(row) > 0:
                row = row.iloc[0]
                print(f"    Q{q}: ${row['Forecast']/1e6:6.1f}M")
        
        # Show FY27 and FY29 samples
        for fy in [2027, 2029]:
            fy_total = team_df[team_df['FiscalYear'] == fy]['Forecast'].sum()
            print(f"  FY{fy} Total: ${fy_total/1e6:.1f}M")
        
        # Calculate CAGR
        fy29_total = team_df[team_df['FiscalYear'] == 2029]['Forecast'].sum()
        fy26_total = team_df[team_df['FiscalYear'] == 2026]['Forecast'].sum()
        cagr = (fy29_total / fy26_total) ** (1/3) - 1
        print(f"  CAGR FY26-29: {cagr*100:.1f}%")
    
    # Export
    export_df = forecast_df[['Team', 'FiscalYear', 'Quarter', 'Forecast', 'StdDev']].copy()
    export_df['Forecast'] = export_df['Forecast'].round(0).astype(int)
    export_df['StdDev'] = export_df['StdDev'].round(0).astype(int)
    
    export_df.to_csv(OUTPUT_FILE, index=False)
    print(f"\nForecasts exported to: {OUTPUT_FILE}")
    
    return forecast_df

# Run the model
if __name__ == "__main__":
    forecast_df = run_forecast_model()
    
    print("\n" + "="*80)
    print("HOW FY26 ASSUMPTIONS IMPROVE THE MODEL:")
    print("="*80)
    print("""
    1. Your FY26 Q2-Q4 inputs create 3 additional YoY data points
    2. These are the MOST RECENT rates, getting highest weight
    3. Example: If you project recovery in Q4, that YoY improvement 
       heavily influences FY27-29 growth projections
    4. This bridges the gap between historical trends and forward expectations
    """)

# ===============================================================
# Create Pivot Tables from forecast_df
# ===============================================================

# Pivot 1: Forecast values
pivot_forecast = (
    forecast_df
    .pivot_table(
        index=['FiscalYear', 'Quarter'],
        columns='Team',
        values='Forecast',
        aggfunc='sum'
    )
    .sort_index()
)

pivot_annual = (
    forecast_df
    .pivot_table(
        index='FiscalYear',
        columns='Team',
        values='Forecast',
        aggfunc='sum'
    )
    .sort_index()
)

# Pivot 2: StdDev values
pivot_stddev = (
    forecast_df
    .pivot_table(
        index=['FiscalYear', 'Quarter'],
        columns='Team',
        values='StdDev',
        aggfunc='sum'
    )
    .sort_index()
)

# Optional: make column order consistent with team_config
team_order = list(team_config.keys())
pivot_forecast = pivot_forecast[team_order]
pivot_annual = pivot_annual[list(team_config.keys())]
pivot_stddev = pivot_stddev[team_order]

# Display
print("\n=== Forecasts by Fiscal Year / Quarter ===")
print(pivot_forecast.round(0).astype(int))

print("\n=== Annual Forecast Totals (Sum of Forecast) ===")
print(pivot_annual.round(0).astype(int))

print("\n=== StdDev by Fiscal Year / Quarter ===")
print(pivot_stddev.round(0).astype(int))


YOY-BASED QUARTERLY FORECASTING MODEL WITH FY26 ASSUMPTIONS

Input file: /Users/matt.fritz/Desktop/SFDC Won Data FY16 pivot.csv
Output file: /Users/matt.fritz/Desktop/quarterly_forecasts_fy26_fy29.csv

Loading historical data...
Successfully loaded data from /Users/matt.fritz/Desktop/SFDC Won Data FY16 pivot.csv
Shape: (41, 10)

CorpFoundation
------------------------------------------------------------

  YoY rates calculated: 37 total
  Most recent YoY rates: ['-2%', '-24%', '-15%', '-1%', '-24%', '-7%', '-8%', '-15%', '-3%', '-21%', '-23%', '-5%', '19%', '56%', '67%', '13%']
  Note: Last 3 rates include FY26 assumptions
  Historical quarterly CV: 40.8%

  FY26 Assumptions:
    Q1 (actual): $23.5M
    Q2 (input):  $21.5M
    Q3 (input):  $14.0M
    Q4 (input):  $8.2M

  Generated growth rates for FY27-29: ['10%', '7%', '11%']
  Weighted historical YoY mean: 7.4%

Marketplace
------------------------------------------------------------

  YoY rates calculated: 37 total
  Most recent Y